<a href="https://colab.research.google.com/github/Bogdan1Gavrilov/dls-homeworks/blob/main/week_09_detection/Homework/homework_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3 style="text-align: center;"><b>Школа глубокого обучения ФПМИ МФТИ</b></h3>

<h3 style="text-align: center;"><b>Домашнее задание. Детекция объектов</b></h3>

В этом домашнем задании мы продолжим работу над детектором из семинара, поэтому при необходимости можете заимствовать оттуда любой код.

Домашнее задание можно разделить на следующие части:

* Переделываем модель [4]
  * Backbone[1],
  * Neck [2],
  * Head [1]
* Label assignment [3]:
  * TAL [3]
* Лоссы [1]:
  * CIoU loss [1]
* Кто больше? [5]
  * 0.05 mAP [1]
  * 0.1 mAP  [2]
  * 0.2 mAP [5]

**Максимальный балл:** 10 баллов. (+3 балла бонус).

In [1]:
import torch
import numpy as np
import pandas as pd
import albumentations as A
import timm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset
from albumentations.pytorch.transforms import ToTensorV2

### Загрузка данных

Мы продолжаем работу с датасетом из семинара - Halo infinite ([сслыка](https://universe.roboflow.com/graham-doerksen/halo-infinite-angel-aim)). Загрузка данных и создание датасета полностью скопированы из семинара.

Сначала загружаем данные

In [2]:
splits = {'train': 'data/train-00000-of-00001-0d6632d599c29801.parquet',
          'validation': 'data/validation-00000-of-00001-c6b77a557eeedd52.parquet',
          'test': 'data/test-00000-of-00001-866d29d8989ea915.parquet'}
df_train = pd.read_parquet("hf://datasets/Francesco/halo-infinite-angel-videogame/" + splits["train"])
df_test = pd.read_parquet("hf://datasets/Francesco/halo-infinite-angel-videogame/" + splits["test"])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Создаем датасет для предобработки данных

В датесете все находится в coco формате, для удобства попробую один раз уницифицировать всё в Pascal и потом не мучаться - надеюсь не сделаю хуже

In [3]:
class HaloDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        df_objects = pd.json_normalize(dataframe['objects'])[["bbox", "category"]]
        df_images = pd.json_normalize(dataframe['image'])[["bytes"]]
        self.data = dataframe[["image_id"]].join(df_objects).join(df_images)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def to_pascal(self, boxes):
        boxes = boxes.copy()
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]  # x_max = x + w
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]  # y_max = y + h
        return boxes

    def __getitem__(self, idx):
        """Загружаем данные и разметку для объекта с индексом `idx`.

        labels: List[int] Набор классов для каждого ббокса,
        boxes: List[List[int]] Набор ббоксов в формате (x_min, y_min, w, h).
        """
        row = self.data.iloc[idx]
        image = Image.open(io.BytesIO(row["bytes"]))
        image = np.array(image)

        target = {}
        target["image_id"] = row["image_id"]

        labels = [row["category"]] if isinstance(row["category"], int) else row['category']
        # Вычитаем единицу чтобы классы начинались с нуля
        labels = [label - 1 for label in labels]
        boxes = row['bbox'].tolist()

        if self.transform is not None:
            transformed = self.transform(image=image, bboxes=boxes, labels=labels)
            image, boxes, labels = transformed["image"], transformed["bboxes"], transformed["labels"]
        else:
            image = transforms.ToTensor()(image)

        target['boxes'] = torch.tensor(np.array(boxes), dtype=torch.float32)
        target['labels'] = torch.tensor(labels, dtype=torch.int64)
        return image, target

def collate_fn(batch):
    batch = tuple(zip(*batch))
    images = torch.stack(batch[0])
    return images, batch[1]

Чтобы модель не переобучалась, можно добавить больше аугментаций, весь список можно посмотреть тут [[ссылка](https://explore.albumentations.ai/)].

Какие можно использовать аугментации?
* Добавить зум `RandomResizedCrop`,
* Сделать цветовые аугментации типа `RandomBrightnessContrast` и/или `HueSaturationValue`,
* Добавить шум `GaussNoise`,
* Вырезать случайные части изображения `CoarseDropout`,
* И любые другие!

Аугментации можно комбинировать посредствам `A.OneOf`, `A.SomeOf` или `A.RandomOrder`.

Хоть аугментации ограничиваются только вашей фантазией, перед обучением советуем посмотреть на результат преобразований и убедиться, что изображение ещё поддается детекции:)

In [4]:
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

train_transform = A.Compose(
    [
        # Добавляй сюда свои аугментации при необходимости!
        A.Normalize(mean=mean, std=std),
        ToTensorV2(),
    ],
    # Раскомментируй, если аугментации изменяют ббоксы.
    # Не забудь указать верный формат для ббоксов.
    # bbox_params=A.BboxParams(format='coco', label_fields=['labels'])
)

test_transform = A.Compose(
    [
        A.Normalize(mean=mean, std=std),
        ToTensorV2(),
    ]
)

Не забываем инициализировать наш датасет

In [5]:
train_dataset = HaloDataset(df_train, transform=train_transform)
test_dataset = HaloDataset(df_test, transform=test_transform)

## Переделываем модель [4 балла]

В семинаре мы реализовали самый базовый детектор, а сейчас настало время его улучшать.

### Backbone [1 балл]

Хорошей практикой считается размораживать несколько последних слоев в backbone, это позволяет немного улучить качество модели. Давайте улушчим класс Backbone из лекции, добавив ему возможность разморозки __k__ последних слоев или блоков (на ваш выбор).

Проверю, точно ли чем меньше индекс тем больше картинки

In [6]:
model = timm.create_model(model_name='efficientnet_b0',pretrained=True,
            features_only=True,
            out_indices=(-3, -2, -1))

for info in model.feature_info.get_dicts():
    print(info)
print(f"Total stages: {len(model.feature_info.get_dicts())}")
x = torch.randn(2, 3, 224, 224)
print(x.shape)
outs = model(x)
for i, out in enumerate(outs):
    print(f'Feature {i}: {out.shape}')
print(f"Total stages: {len(model.feature_info.get_dicts())}")

model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

{'stage': 3, 'reduction': 8, 'module': 'blocks.2', 'num_chs': 40, 'index': 2}
{'stage': 5, 'reduction': 16, 'module': 'blocks.4', 'num_chs': 112, 'index': 3}
{'stage': 7, 'reduction': 32, 'module': 'blocks.6', 'num_chs': 320, 'index': 4}
Total stages: 3
torch.Size([2, 3, 224, 224])
Feature 0: torch.Size([2, 40, 28, 28])
Feature 1: torch.Size([2, 112, 14, 14])
Feature 2: torch.Size([2, 320, 7, 7])
Total stages: 3


In [7]:
class Backbone(nn.Module):
    def __init__(self, model_name='efficientnet_b0', out_indices=(-3, -2, -1), unfreeze_last_layers=3):
        super().__init__()
        self.backbone = timm.create_model(
            model_name=model_name,
            pretrained=True,
            features_only=True,
            out_indices=out_indices
        )

        # Замораживаем всё
        for param in self.backbone.parameters():
            param.requires_grad = False
        block_names = []
        for name, module in self.backbone.named_modules():
            # берем только блоки верхнего уровня (типа 'blocks.0', 'blocks.1', ...)
            if name.startswith('blocks.') and name.count('.') == 1:
                block_names.append(name)
        # Размораживаем только последние unfreeze_last_layers слоёв
        to_unfreeze = block_names[-unfreeze_last_layers:]
        for name, module in self.backbone.named_modules():
            if name in to_unfreeze:
                for param in module.parameters():
                      param.requires_grad = True

    def forward(self, x):
        return self.backbone(x)

### NECK [2 балла]

Следующее улучшение коснется шеи. Предлагаем реализовать знакомую из лекции архитектуру FPN.

#### Feature Pyramid Network

<center><img src="https://user-images.githubusercontent.com/57972646/69858594-b14a6c00-12d5-11ea-8c3e-3c17063110d3.png"/></center>


* [Feature Pyramid Networks for Object Detection](https://arxiv.org/abs/1612.03144)

Она состоит из top-down пути, в котором происходит 2 вещи:
1. Увеличивается пространственная размерность фичей,
2. С помощью скипконнекшеннов, добавляются фичи из backbone модели.

Для увеличения пространственной размерности используется __nearest neighbor upsampling__, а фичи из шеи и бекбоуна суммируются.

__TIPS__:
* Можете использовать базовые классы из лекции,
* Воспользуйтесь AnchorGenerator-ом, чтобы создавать якоря сразу для нескольких выходов,
* Не забудьте использовать nn.ModuleList, если захотите сделать динамическое количество голов у модели,
* Также, можно добавить доп конволюцию (3х3 с паддингом) у каждого выхода шеи.

фичи представляют собой список длиной 3 -> in_channels тоже будет списком длиной 3, а вот out_channels единым можно сделать спокойно

In [8]:
import torch.nn as nn

class Neck(nn.Module):
    def __init__(self, in_channels, out_channels, use_activations=True):
        super().__init__()
        self.num_ch = len(in_channels)
        self.skips = nn.ModuleList()
        self.outlayers = nn.ModuleList()
        self.bns = nn.ModuleList([nn.BatchNorm2d(out_channels) for _ in in_channels])
        self.activation = nn.LeakyReLU(inplace=True) if use_activations else nn.Identity()

        self.upsample = nn.Upsample(mode='nearest', scale_factor=2)

        for channels in in_channels:
            self.skips.append(nn.Conv2d(channels, out_channels, kernel_size=1))
            self.outlayers.append(nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1))

    def forward(self, features):
        skips = []
        for i in range(len(features)):
            skips.append(self.skips[i](features[i]))

        levels = [None] * len(skips) #список сложенных апсмепленных и более низкоуровневых фичей
        levels[-1] = skips[-1] #последний равен, потому что к нему прибавлять нечего

        for i in range(len(skips) - 2, -1, -1):
            up = self.upsample(levels[i + 1])
            levels[i] = skips[i] + up

        output = []
        for i in range(len(skips)):
            out = self.outlayers[i](levels[i])
            out = self.bns[i](out)
            out = self.activation(out)
            output.append(out)

        return output

### Head [1 балл]

В качестве шеи можно выбрать __один из двух__ вариантов:

#### 1. Decoupled Head

Реализовать Decoupled Head из [YOLOX](https://arxiv.org/abs/2107.08430).
<center><img src="https://i.ibb.co/BVtBR2R3/Decoupled-head.jpg"/></center>

**TIP**: Возьмите за основу голову из семинара, тк она сильно похожа на Decoupled Head.

Изменять количество параметров у шей на разных уровнях не обязательно.

#### 2. Confidence score free head

Нужно взять за основу голову из семинара и полностью убрать предсказание confidence score. Чтобы модель предсказывала только 2 группы: ббоксы и классы.

Есть следующие способы удаления confidence score:
* Добавление нового класса ФОН. Обычно его обозначают нулевым классом.
* Присваивание ббоксам БЕЗ объекта вектор из нулей в качестве таргета.

Выберете тот, который вам больше нравится и будте внимательны при расчете лосса!

**Важно!** Удаление confidence score повлияет на следующие методы из семинара:
* target_assign
* ComputeLoss
* _filter_predictions

Голова снова получает список из фичей трех различных размеров

регрессионная и IoU часть проходят по общей main части, только потом делятся

In [11]:
from torch.nn.modules.activation import LeakyReLU
class Head(nn.Module):
    def __init__(self, in_channels, num_anchors, num_classes):
        super().__init__()
        self.firstconv = nn.Conv2d(in_channels, in_channels, 1)
        self.cls_branch = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, 3, padding=1,),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels, in_channels, 3, 1),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels, num_classes*num_anchors, 1)
        )
        self.main_branch = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, in_channels, 3, padding=1),
            nn.ReLU(inplace=True),
        )
        self.reg_last = nn.Conv2d(in_channels, num_anchors * 4, 1)

        self.iou_last = nn.Conv2d(in_channels, num_anchors, 1)

    def forward(self, features):
        # features — список тензоров от Neck
        cls_outs, reg_outs, iou_outs = [], [], []
        for feat in features:
            feat = self.firstconv(feat)
            cls_outs.append(self.cls_branch(feat))
            main_feat = self.main_branch(feat)
            reg_outs.append(self.reg_last(main_feat))
            iou_outs.append(self.iou_last(main_feat))
        return cls_outs, reg_outs, iou_outs  # списки для каждого уровня

Теперь можно снова реализовать класс детектора с учетом всех частей выше!

In [13]:
anchor_sizes=((32, 64), (64, 128), (128, 256))
anchor_ratios=((0.5, 1.0, 2.0),) * 3
len(anchor_sizes[0]) * len(anchor_ratios)

6

In [14]:
class Detector(nn.Module):
    """ Класс детектора, объединяющий все части вместе.

    Параметры
    ---------
    backbone_model_name : str
        Имя timm модели, которая будет исопльзоваться в качестве backbone
    neck_n_channels : int
        Итоговое количество каналов на выходе шеи
    num_classes : int
        Количество классов в датасете
    anchor_sizes : list, tuple
        Базовые размеры якорей. Используется методом `AnchorDetector` из
        `torchvision.models.detection.anchor_utils`для создания якорей.
    anchor_ratios : list, tuple
        Соотношения сторон якорей. Используется методом `AnchorDetector` из
        `torchvision.models.detection.anchor_utils`для создания якорей.
    input_size : tuple with length 2
        Размер входного изображения. Используется для расчета положения всех якорей.

    NOTE: На самом деле, AnchorDetector на вход ждет anchor_sizes и anchor_ratios как Tuple[Tuple[int]],
    это связано с возможностью создания разных якорей для разных выходов модели. Сейчас нам это не нужно,
    поэтому структура намеренно упрощена.
    """
    def __init__(self,
                 backbone_model_name="efficientnet_b0",
                 neck_n_channels=256,
                 num_classes=4,
                 anchor_sizes=((32, 64), (64, 128), (128, 256)),  # ну раз туплю туплей хотят то попробую так
                 anchor_ratios=((0.5, 1.0, 2.0),) * 3,
                 input_size=(640, 640),
        ):
        super().__init__()
        self.num_classes = num_classes
        # Создаем backbone c тремя выходами
        # Если нужно сделать несколько выходов, меняй out_indices
        self.backbone = Backbone(backbone_model_name, out_indices=(-3, -2, -1))
        # Получаем количество фичемапов на выходном слое
        in_channels = self.backbone.backbone.feature_info.channels()
        # Создаем шею, с учетом параметров, полученных выше
        self.neck = Neck(in_channels, out_channels=neck_n_channels)
        # Размерность выходов головы зависит от количество якорей, поэтому сначала считаем количество якорей в каждой
        # ячейке выходной фичемапы
        num_anchors = len(anchor_sizes[0]) * len(anchor_ratios)
        self.head = Head(in_channels=neck_n_channels, num_anchors=num_anchors, num_classes=num_classes)

       # Генерируем якоря
        anchor_generator = AnchorGenerator(sizes=anchor_sizes, aspect_ratios=anchor_ratios)
        reductions = self.backbone.backbone.feature_info.reduction()
        anchors_list = []

        for reduction in reductions:
            grid_size = [input_size[0] // reduction, input_size[1] // reduction]
            anchors = anchor_generator.grid_anchors([grid_size], strides=[[reduction, reduction]])
            anchors_list.append(anchors[0])  # [H*W*num_anchors, 4]

        # ✅ КРИТИЧЕСКИ ВАЖНО: регистрируем anchors как buffers
        all_anchors = torch.cat(anchors_list, dim=0)  # [total_anchors, 4]
        self.register_buffer('all_anchors', all_anchors)

        # Вычисляем центры и размеры anchor'ов
        anchor_centers = torch.stack([
            (all_anchors[:, 0] + all_anchors[:, 2]) / 2,  # center_x
            (all_anchors[:, 1] + all_anchors[:, 3]) / 2   # center_y
        ], dim=1)

        anchor_sizes = torch.stack([
            all_anchors[:, 2] - all_anchors[:, 0],  # width
            all_anchors[:, 3] - all_anchors[:, 1]   # height
        ], dim=1)

        self.register_buffer('anchor_centers', anchor_centers)
        self.register_buffer('anchor_sizes', anchor_sizes)

    def forward(self, x):
        features = self.backbone(x)
        neck_features = self.neck(features)
        cls_logits_list, bbox_preds_list, iou_preds_list = self.head(neck_features)

        N = x.shape[0]

        # Обрабатываем каждый уровень FPN и конкатенируем
        all_cls_logits = []
        all_bbox_preds = []
        all_iou_preds = []

        for cls_logits, bbox_preds, iou_preds in zip(cls_logits_list, bbox_preds_list, iou_preds_list):
            # cls_logits: [N, num_anchors * num_classes, H, W]
            # bbox_preds: [N, num_anchors * 4, H, W]
            # iou_preds: [N, num_anchors, H, W]

            # Reshape classification
            cls_reshaped = cls_logits.permute(0, 2, 3, 1).contiguous()
            cls_reshaped = cls_reshaped.view(N, -1, self.num_classes)
            all_cls_logits.append(cls_reshaped)

            # Reshape bbox predictions
            bbox_reshaped = bbox_preds.permute(0, 2, 3, 1).contiguous()
            bbox_reshaped = bbox_reshaped.view(N, -1, 4)
            all_bbox_preds.append(bbox_reshaped)

            # Reshape IoU predictions
            iou_reshaped = iou_preds.permute(0, 2, 3, 1).contiguous()
            iou_reshaped = iou_reshaped.view(N, -1, 1)
            all_iou_preds.append(iou_reshaped)

        # Конкатенируем все уровни
        cls_logits = torch.cat(all_cls_logits, dim=1)  # [N, total_anchors, num_classes]
        bbox_preds = torch.cat(all_bbox_preds, dim=1)   # [N, total_anchors, 4]
        iou_preds = torch.cat(all_iou_preds, dim=1)     # [N, total_anchors, 1]

        confidence_logits = iou_preds.squeeze(-1)  # [N, total_anchors]

        if self.training:
            return bbox_preds, cls_logits, confidence_logits

        # Inference
        bboxes = self.decode_bboxes(bbox_preds)  # [N, total_anchors, 4] в формате xyxy
        confidence = torch.sigmoid(confidence_logits)
        cls_probs = torch.softmax(cls_logits, dim=-1)

        return bboxes, confidence, cls_probs


    def decode_bboxes(self, bbox_offsets):
        """
        Декодирование предсказанных смещений в абсолютные координаты xyxy.
        Предполагаем YOLO-style параметризацию.
        """
        # bbox_offsets: [N, total_anchors, 4] - (tx, ty, tw, th)

        # Expand anchor buffers to batch dimension
        anchor_centers = self.anchor_centers.unsqueeze(0).expand(bbox_offsets.size(0), -1, -1)  # [N, total_anchors, 2]
        anchor_sizes = self.anchor_sizes.unsqueeze(0).expand(bbox_offsets.size(0), -1, -1)      # [N, total_anchors, 2]

        tx = bbox_offsets[..., 0]
        ty = bbox_offsets[..., 1]
        tw = bbox_offsets[..., 2]
        th = bbox_offsets[..., 3]

        # Декодирование центров (YOLO-style с сигмоидой)
        center_x = anchor_centers[..., 0] + torch.sigmoid(tx) * anchor_sizes[..., 0]
        center_y = anchor_centers[..., 1] + torch.sigmoid(ty) * anchor_sizes[..., 1]

        # Декодирование размеров
        w = anchor_sizes[..., 0] * torch.exp(tw)
        h = anchor_sizes[..., 1] * torch.exp(th)

        # Конвертация в xyxy
        x_min = center_x - w / 2
        y_min = center_y - h / 2
        x_max = center_x + w / 2
        y_max = center_y + h / 2

        return torch.stack([x_min, y_min, x_max, y_max], dim=-1)

ВОТ ЭТО ПРОСТО УЖАС, НЕСКОЛЬКО ЧАСОВ СИДЕЛ НАД ЭТИМ С ОЩУЩЕНИЕМ ЧТО БЬЮСЬ ГОЛОВОЙ ОБ СТЕНУ БЕЗОСТАНОВОЧНО

## Label assignment [3 балла]
В этой секции предлагается заменить функцию `assign_target` на более современный алгоритм который называется Task alignment learning.

Он описан в статье [TOOD](https://arxiv.org/abs/2108.07755) в секции 3.2. Для удобства вот его основные шаги:

1. Посчитать значение метрики для каждого предсказанного ббокса:
    
$$t = s^\alpha * u^\beta$$
    
где,
* $s$ — classification score, или вероятность принадлежности предсказанного ббокса к классу реального ббокса (**GT**);
* $u$ — IoU между предсказанным и реальным ббоксами;
* $\alpha,\ \beta$ — нормализационные константы, обычно $\alpha = 6.0, \ \beta = 1.0$.
    
2. Отфильтровать предсказания на основе **GT**.

    Для якорных детекторов, обычно, выбираются только те предсказания, центры якорей которых находятся внутри GT.
4. Для каждого **GT** выбрать несколько (обычно 5 или 13) самых подходящих предсказаний.
5. Если предсказание рассматривается в качестве подходящего для нескольких **GT** — выбрать **GT** с наибольшим пересечением по IoU.


**BAЖНО**: если будете использовать Runner из лекции, не забудьте поменять параметры  в `self.assign_target_method` в методе `_run_train_epoch`.

In [ ]:
class TAL_assigner(nn.Module):
    def __init__(self, topk=13, alpha=1.0, beta=6.0, eps=1e-8):
        super().__init__()
        self.topk = topk
        self.alpha = alpha
        self.beta = beta
        self.eps = eps

    @torch.no_grad()
    def forward(self, pred_bboxes, pred_scores, anchors, gt_bboxes, gt_labels):
        """
        Args:
            pred_bboxes: [N, total_anchors, 4] - предсказанные bbox в формате xyxy
            pred_scores: [N, total_anchors, num_classes] - предсказанные scores
            anchors: [total_anchors, 4] - анкоры в формате xyxy
            gt_bboxes: list of [num_gt, 4] - GT bbox для каждого изображения
            gt_labels: list of [num_gt] - GT labels для каждого изображения

        Returns:
            assigned_labels: [N, total_anchors] - назначенные метки
            assigned_bboxes: [N, total_anchors, 4] - целевые bbox
            pos_mask: [N, total_anchors] - маска позитивных анкоров
        """
        batch_size = pred_bboxes.shape[0]
        num_anchors = pred_bboxes.shape[1]
        device = pred_bboxes.device

        assigned_labels = torch.full((batch_size, num_anchors),
                                   -1, dtype=torch.long, device=device)
        assigned_bboxes = torch.zeros_like(pred_bboxes)
        pos_mask = torch.zeros((batch_size, num_anchors), dtype=torch.bool, device=device)

        for i in range(batch_size):
            if len(gt_bboxes[i]) == 0:
                continue

            # 1. Вычисляем IoU между предсказаниями и GT
            ious = self.bbox_iou(pred_bboxes[i], gt_bboxes[i])  # [num_anchors, num_gt]

            # 2. Вычисляем Task-Aligned Metric
            gt_scores = pred_scores[i][:, gt_labels[i]]  # [num_anchors, num_gt]
            alignment_metric = gt_scores.pow(self.alpha) * ious.pow(self.beta)

            # 3. Назначаем топ-k анкоров для каждого GT
            topk_metrics, topk_indices = alignment_metric.topk(
                self.topk, dim=0, largest=True, sorted=False)

            # 4. Создаем маски назначения
            for gt_idx in range(len(gt_bboxes[i])):
                anchor_indices = topk_indices[:, gt_idx]
                assigned_labels[i, anchor_indices] = gt_labels[i][gt_idx]
                assigned_bboxes[i, anchor_indices] = gt_bboxes[i][gt_idx].unsqueeze(0)
                pos_mask[i, anchor_indices] = True

        return assigned_labels, assigned_bboxes, pos_mask

    def bbox_iou(self, box1, box2):
        """Compute IoU between two sets of boxes in xyxy format"""
        # box1: [M, 4], box2: [N, 4]
        area1 = (box1[:, 2] - box1[:, 0]) * (box1[:, 3] - box1[:, 1])
        area2 = (box2[:, 2] - box2[:, 0]) * (box2[:, 3] - box2[:, 1])

        lt = torch.max(box1[:, None, :2], box2[:, :2])
        rb = torch.min(box1[:, None, 2:], box2[:, 2:])

        wh = (rb - lt).clamp(min=0)
        inter = wh[:, :, 0] * wh[:, :, 1]

        iou = inter / (area1[:, None] + area2 - inter + self.eps)
        return iou

### DIoU [1]

Вместо SmoothL1, который используется в семинаре, реализуем лосс, основанный на пересечении ббоксов. В качестве тренировки давайте напишем Distance Intersection over Union (DIoU).

<center><img src=https://wikidocs.net/images/page/163613/Free_Fig_5.png></center>

Для его реализации разобъем задачу на части:

**1. Реализуем IoU:**

Пусть даны координаты для предсказанного ($B^p$) и истинного ($B^g$) ббоксов в формате XYXY или VOC PASCAL (левый верхний и правый нижний углы):

$B^p=(x^p_1, y^p_1, x^p_2, y^p_2)$, $B^g=(x^g_1, y^g_1, x^g_2, y^g_2)$, тогда алгоритм расчета будет следующий:

    1. Найдем площади обоих ббоксов:
$$ A^p = (x^p_2 - x^p_1) * (y^p_2 - y^p_1) $$
$$ A^g = (x^g_2 - x^g_1) * (y^g_2 - y^g_1) $$

    2. Посчитаем пересечение между ббоксами:

Тут мы предлагаем вам подумать как в общем виде можно расчитать размеры ббокса, который будет являться пересечением $B^p$ и $B^g$, а затем посчитать его площадь:

$$x^I_1 = \qquad \qquad y^I_1 = $$
$$x^I_2 = \qquad \qquad y^I_2 = $$

В общем виде, площать будет записываться следующим образом:

Если $x^I_2 > x^I_1$ & $y^I_2 > y^I_1$, тогда:

$$I = (x^I_2 - x^I_1) * (y^I_2 - y^I_1)$$

Иначе, $I = 0$.

    3. Считаем объединение ббоксов.

Мы можем посчитать эту площадь как сумму площадей двух ббоксов минус площадь пересечения (тк мы считаем её два раз в сумме площадей):

$$U = A^p + A^g - I$$

    4. Вычисляем IoU.

$$IoU = \frac{I}{U}$$

**2. Посчитаем диагональ выпуклой оболочки:**

Для расчета диагонали, сначала выпишите координаты верхнего левого и правого нижнего углов. Подумайте, чему будут равны эти координаты в общем случае?

$$x^c_1 = \qquad \qquad y^c_1 = $$
$$x^c_2 = \qquad \qquad y^c_2 = $$

Подсказка: Нарисуйте несколько вариантов пересечений предсказания и GT на бумажке, и выпишите координаты для выпуклой оболочки.

Тогда квадрат диагонали можно посчитать по формуле:

$$c^2 = (x^c_2 - x^c_1)^2 + (y^c_2 - y^c_1)^2$$

**3. Рассчитаем расстояние между цетрами ббоксов:**

Сначала находим координаты центров каждого из ббоксов (если ббоксы в формате YOLO, то и считать ничего не нужно), затем считаем Евклидово расстояние между центрами.

$d = $

Собираем все части вместе и считаем лосс по формуле:

$$ DIoU = 1 - IoU + \frac{d^2}{c^2}$$

Помните, что пар ббоксов может быть много! Возвращайте усредненное значение лосса.

In [ ]:
from torchvision.ops import distance_box_iou_loss

In [ ]:
def gen_bbox(num_boxes=10):
    min_corner = torch.randint(0, 100, (num_boxes, 2))
    max_corner = torch.randint(50, 150, (num_boxes, 2))

    for i in range(2):
        wrong_order = min_corner[:, i] > max_corner[:, i]
        if wrong_order.any():
            min_corner[wrong_order, i], max_corner[wrong_order, i] = max_corner[wrong_order, i], min_corner[wrong_order, i]
    return torch.cat((min_corner, max_corner), dim=1)

In [ ]:
pred_boxes = gen_bbox(num_boxes=100)
true_boxes = gen_bbox(num_boxes=100)

In [ ]:
print(f" DIoU: {distance_box_iou_loss(pred_boxes, true_boxes, reduction="mean").item()}")

In [ ]:
def diou_loss(pred_boxes, gt_boxes, eps=1e-8):
    S_pr = (pred_boxes[:, 3] - pred_boxes[:, 1]) * (pred_boxes[:, 2] - pred_boxes[:, 0])
    S_gt = (gt_boxes[:, 3] - gt_boxes[:, 1]) * (gt_boxes[:, 2] - gt_boxes[:, 0])

    inter_xmin = torch.max(pred_boxes[:, 0], gt_boxes[:, 0])
    inter_ymin = torch.max(pred_boxes[:, 1], gt_boxes[:, 1])
    inter_xmax = torch.min(pred_boxes[:, 2], gt_boxes[:, 2])
    inter_ymax = torch.min(pred_boxes[:, 3], gt_boxes[:, 3])

    S_inter = (inter_xmax - inter_xmin) * (inter_ymax - inter_ymin)
    Union = S_pr + S_gt - S_inter
    IoU = S_inter / Union

    return

In [ ]:
import numpy as np
pred_boxes = gen_bbox(num_boxes=1000)
true_boxes = gen_bbox(num_boxes=1000)

# проверим что написанный лосс выдает те же результаты что и лосс из торча.
assert np.isclose(diou_loss(pred_boxes, true_boxes), distance_box_iou_loss(pred_boxes, true_boxes, reduction="mean"))

## Кто больше? [5 баллов]

Наконец то мы дошли до самый интересной части. Тут мы раздаем очки за mAP'ы!

Все что вы написали выше вам поможет улучшить качество итогового детектора, настало время узнать насколько сильно :)

За достижения порога по mAP на тестовом наборе вы получаете баллы:
* 0.05 mAP [1]
* 0.1 mAP [2]
* 0.2 mAP [5]


**TIPS**:
1. На семинаре мы специально не унифицировали формат ббоксов между методами, чтобы обратить ваше внимание что за этим нужно следить. Чтобы было проще, сразу унифицируете формат по всему ноутбуку. Советуем использовать формат xyxy, тк IoU и NMS из torch используют именно этот формат. (Не забудьте поменять формат у таргета в `HaloDataset`).

2. Попробуйте перейти к IoU-based лоссу при обучении. То есть обучать не смещения, а сразу предсказывать ббокс.

3. Поэксперементируйте с подходами target assignment'а в процессе обучения. Например, можно на первых итерациях использовать обычный метод, а затем подключить TAL.

4. Добавьте аугментаций!

Можно взять [albumentations](https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/), библиотеку, которую мы использовали всеминаре. Или базовые аугментации из торча [тык](https://pytorch.org/vision/main/transforms.html). Если будете использовать торч, не забудте про ббоксы, transforms из коробки не будет их агументировать.

5. Можете реализовать другую шею, которую мы обсуждали на лекции [Path Aggregation Network](https://arxiv.org/abs/1803.01534) она точно улучшит ваше итоговое качество.

6. Попробуйте добавлять различные блоки из YOLO архитектур в шею вместо единичных конволюционных слоев. (Например, замените конволюции 3х3 на CSP блоки).

7. Попробуйте заменить NMS на другой метод (WeightedNMS, SoftNMS, etc.). Немного ссылок:
    * Статья про SoftNMS [тык](https://arxiv.org/pdf/1704.04503)
    * Статья про WeightedNMS [тык](https://openaccess.thecvf.com/content_ICCV_2017_workshops/papers/w14/Zhou_CAD_Scale_Invariant_ICCV_2017_paper.pdf)
    * Есть их реализация, правда на нумбе [git](https://github.com/ZFTurbo/Weighted-Boxes-Fusion?tab=readme-ov-file)

8. Не бойтесь эксперементировать и удачи!

Также, напишите развернутые ответы на следующие вопросы:

**Questions:**
1. Какой метод label assignment'a помогает лучше обучаться модели? Почему?
2. Какое из сделаных вами улучшений внесло наибольший вклад в качество модели? Как вы думаете, почему это произошло?
3. Какое из сделанных вами улучшений вообще не изменило метрику? Как вы думаете, почему это произошло?

In [ ]:

# Место для обучения, валидации и экспериментов!


Ниже определена вспомогательная функция для валидации качества. Можете использовать `Runner.validate`. Важное уточнение, ей нужен метод для фильтрации предсказаний. Можете тоже скопировать его из семинара, если он у вас не менялся.

In [ ]:
from torchmetrics.detection import MeanAveragePrecision

@torch.no_grad()
def validate(dataloader, filter_predictions_func, box_format="xyxy", device="cpu", score_threshold=0.1, nms_threshold=0.5, **kwargs):
    """ Метод для валидации модели.
    Возвращает mAP (0.5 ... 0.95).
    """
    self.model.eval()
    # Считаем метрику mAP с помощью функции из torchmetrics
    metric = MeanAveragePrecision(box_format=box_format, iou_type="bbox")
    for images, targets in tqdm(dataloader, desc="Running validation", leave=False):
        images = images.to(device)
        outputs = self.model(images)
        predicts = filter_predictions_func(outputs, score_threshold, nms_threshold, **kwargs)
        metric.update(predicts, targets)
    return metric.compute()["map"].item()
